![CC-BY-SA](https://mirrors.creativecommons.org/presskit/buttons/88x31/svg/by-sa.svg)
This notebook was created by [Bernardo Freitas Paulo da Costa](http://www.im.ufrj.br/bernardofpc),
and is licensed under Creative Commons BY-SA

# Vamos melhorar a P2

O objetivo é fazer o gráfico da primitiva de várias funções,
calculando-as o mínimo possível.

## Funções que se contam

Para ver, sem dúvidas, quantas vezes uma função foi chamada,
o mais simples é que ela mesma modifique um contador cada vez que ela é chamada.
No nosso caso, com funções vetorizadas do `numpy`, temos que levar em conta duas "medidas":
- cada vez que a função é chamada,
- o número de pontos em que ela foi calculada.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Adaptado de http://www.python-course.eu/python3_count_function_calls.php
def counted(fn):
    def wrapper(xs, *args, **kwargs):
        wrapper.called += 1
        if isinstance(xs, np.ndarray):
            wrapper.nevals += xs.size
        elif isinstance(xs, list):
            wrapper.nevals += len(xs)
        else:
            wrapper.nevals += 1
        return fn(xs, *args, **kwargs)
    wrapper.called = 0
    wrapper.nevals = 0
    wrapper.__name__= fn.__name__
    return wrapper

### Exemplo de uso de `counted`:

In [ ]:
seno = counted(np.sin)

In [ ]:
a = np.random.rand(10,30)

In [ ]:
_ = seno(a)
seno.called, seno.nevals

In [ ]:
_ = seno([4,4,5])
seno.called, seno.nevals

# Parte 3

Use `np.cumsum` para evitar calcular várias vezes as funções num mesmo ponto:

In [ ]:
def prim_cauchy(f,a,b,n):
    """Retorna a primitiva de $f$ no intervalo $[a,b]$, calculado em $n$ pontos intermediários via Cauchy."""
    ### Resposta aqui


In [ ]:
# Reset
seno.called = 0
seno.nevals = 0
# Cálculo
ts, pri = prim_cauchy(seno, 0, 10, 230)
# Esperado: (1, 230)
seno.called, seno.nevals

In [ ]:
plt.plot(ts, pri)
plt.plot(ts, 1 - np.cos(ts))
plt.show()

Faça o mesmo para `prim_midpoint` e `prim_simpson`.
No caso de Simpson, se você chamar apenas duas vezes a função (com dois vetores) já está bastante bom.

In [ ]:
def prim_midpoint(f,a,b,n):
    ### Resposta aqui


In [ ]:
def prim_simpson(f,a,b,n):
    ### Resposta aqui


In [ ]:
prim_simpson(seno, 0, 10, 10)

Faça agora um gráfico do erro em função de `nevals`, ao mudar $n$ nos três métodos acima.

Sugestão: faça três eixos, um para cada método, mas faça várias curvas de erro em cada eixo.
Tente também comparar erros para o mesmo valor de `nevals`.

In [ ]:
eval_per_int = [1,1,2]
meths = [prim_cauchy, prim_midpoint, prim_simpson]
names = ['cauchy', 'midpoint', 'simpson']

In [ ]:
### Defina aqui uma função para fazer as contas
### Resposta aqui


In [ ]:
### Gráficos aqui
### Resposta aqui


Observe o que acontece para outras funções, por exemplo
- `np.exp`
- `np.log`
- ...

Uma medida interessante é o erro _relativo_ cometido, não apenas o erro absoluto.

In [ ]:
expo = counted(np.exp)

In [ ]:
### Resposta aqui


In [ ]:
### Resposta aqui


# Parte 4

Agora, vamos calcular interpoladores também mais eficientes.
A estratégia será a seguinte:
- Escolher uma "ordem" de interpolação $n$, e um grau de "sobreposição" $m$;
- Obter uma lista de polinômios interpoladores, em grupos sucessivos de pontos calculados via `cumsum` da parte anterior;
- Usar `np.piecewise` para escolher qual polinômio usar, para cada ponto em que desejamos aproximar a primitiva.

## Interpolador por partes, com sobreposição

Dados dois números $n > m$, separe os nós `ts` em grupos de $n$ pontos,
onde grupos sucessivos contém $m$ pontos em comum.
Vamos tentar observar as vantagens e desvantagens de tomar $m$ grande.

In [ ]:
def particionar(xs,n,m):
    """Particiona os nós de interpolação `xs` em grupos de n nós, com sobreposição de m,
    retornando a lista de grupos.
    
    Se necessário, o último grupo pode ter menos de n nós, mas sempre terá mais do que m nós."""
    # Você pode usar que xs[i:j] nunca dá erro: o "pior" que pode acontecer é retornar um array vazio.
    assert n > m >= 0
    N = len(xs)
    ### Resposta aqui


In [ ]:
particionar(np.arange(19), 5, 2)

In [ ]:
particionar(np.arange(20), 5, 2)

Agora, com a mesma ideia de particionar, divida as listas de pontos de interpolação em grupos:

In [ ]:
def agrupar(xs, ys, n, m):
    assert n > m >= 0
    N = len(xs)
    ### Resposta aqui


In [ ]:
xs = np.arange(20)
agrupar(xs, np.sin(xs), 5, 2)

Verifique que a sua função agrupadora funciona com argumentos que não são inteiros consecutivos.
Por exemplo, faça testes com `rand`...

## Fatiando os intervalos

Agora que temos diversos intervalos disponíveis para interpolar,
precisamos decidir a qual dos intervalos pertencem os números de um vetor `ts`.
Escreva uma função que, dados $xs$, $n$ e $m$, retorna duas listas,
com extremidades inferiores e superiores de interpolação.

Obs: na verdade, como vamos usar `np.piecewise()`,
não é necessário retornar exatamente extremidades disjuntas.
Isso pode simplificar o código, como na função abaixo.

In [ ]:
def extremidades_1(xs,n,m):
    """Exemplo de cálculo das extremidades."""
    grupos = particionar(xs,n,m)
    return [g[0] for g in grupos], [g[-1] for g in grupos]

In [ ]:
extremidades_1(xs,5,2)

Depois de fazer as questões seguintes usando este cálculo "simples" de extremidades,
defina uma versão melhor abaixo, e explique porque ela deveria ser melhor.

In [ ]:
def extremidades(xs,n,m):
    ### Resposta aqui


Explique aqui ;-)

## A função interpoladora por partes

Agora, com a lista de pontos para interpolar, e as extremidades que "escolhem" cada polinômio,
- calcule (uma única vez) os polinômios interpoladores;
- retorne uma função (vetorial!) que usa `np.piecewise()` e as extremidades para calcular a interpolação por partes.

In [ ]:
from interpolation import baricentric as lagrange

In [ ]:
def build_interp(xs, ys, n, m):
    """Retorna uma lista de polinômios interpoladores usando divisões em blocos de n pontos."""
    blocos = agrupar(xs,ys,n,m)
    polys = [lagrange(xis,yis) for xis,yis in blocos]
    l,r = extremidades(xs,n,m)
    def ipp(ts):
        conds = [np.all([li <= ts, ts <= ri], axis=0) for li,ri in zip(l,r)]
        return np.piecewise(ts, conds, polys)
    return ipp

### Um teste do interpolador

In [ ]:
xs = np.arange(0,5,0.1)
ys = np.sin(xs)
f = build_interp(xs,ys,6,2)
# Em 1 ponto
f(np.pi)

In [ ]:
# Em 1 array
t = np.random.rand(10)
print(f(t))
print(np.sin(t))
print(f(t) - np.sin(t))

# Juntando com as primitivas

## Um primeiro exemplo

A partir dos pontos abaixo,
observe como a escolha dos valores de "ordem" e "sobreposição" ($n$ e $m$)
influenciam a qualidade da interpolação para pontos "no meio".

In [ ]:
### Pontos calculados via integração numérica + cumsum
xs, Fxs = prim_simpson(seno, 0, 10, 200)
### Pontos a calcular por interpolação
ts = np.linspace(3,5,num=300)

In [ ]:
### Resposta aqui


In [ ]:
### Aumente e diminua a ordem de interpolação.  Até onde devemos ir?
### Resposta aqui


## Outros exemplos

Repita o estudo acima para outras funções, tanto com primitivas conhecidas, como com primitivas desconhecidas.
Qual a melhor forma de estimar uma primitiva com alta precisão?

# Diversão extra

É claro que calcular integrais precisas com a regra de Simpson não é o mais eficiente...
Tente usar `np.cumsum()` também para os métodos interpolatórios de alta ordem,
e tente calcular (por exemplo) a primitiva de $f(x) = \sin(x) + \cos(\pi x)$
com precisão de `1e-12` ao longo do intervalo $[0,20]$, calculando o mínimo possível a função $f$.